# Testing GPT2 on spongebob scripts 

Credit to the work done by Mikhail Gaerlan and Max Woolf

# Installation and Asserting Version Requirements

In [ ]:

from platform import python_version
print(python_version())

3.7.10


In [ ]:

%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from glob import glob



TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
!nvidia-smi

Sun Apr 11 00:48:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading GPT2


* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 402Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 650kit/s]
Fetching hparams.json: 1.05Mit [00:00, 330Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [06:49, 1.22Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 312Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:01, 980kit/s]
Fetching vocab.bpe: 1.05Mit [00:01, 941kit/s]


# Uploading text file for training

In [ ]:
# import os
# os.chdir("drive/MyDrive/w266 Final Project Resources")
# os.listdir()



In [ ]:

file_name = "spongebob_updated_with_FP.txt"



In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
sess = gpt2.start_tf_sess()


In [ ]:
#gpt2.load_gpt2(sess)

In [ ]:

gpt2.finetune(sess,
              dataset=file_name,
              restore_from ='fresh',
              model_name='124M',
              steps=500,
              print_every=50,
              sample_every=500,
              save_every=100,
             
              batch_size=2

             
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.30s/it]


dataset has 2139121 tokens
Training...
[50 | 125.07] loss=2.54 avg=2.54
[100 | 243.59] loss=2.52 avg=2.53
Saving checkpoint/run1/model-100
[150 | 364.46] loss=2.52 avg=2.53
[200 | 483.02] loss=2.44 avg=2.51
Saving checkpoint/run1/model-200
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[250 | 603.43] loss=2.22 avg=2.45
[300 | 722.00] loss=2.18 avg=2.40
Saving checkpoint/run1/model-300
[350 | 842.33] loss=2.22 avg=2.38
[400 | 960.93] loss=2.18 avg=2.35
Saving checkpoint/run1/model-400
[450 | 1081.36] loss=1.90 avg=2.30
[500 | 1199.95] loss=1.88 avg=2.25
Saving checkpoint/run1/model-500


In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-500
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-500


In [ ]:
for i in range(1,500):
  gpt2.generate(sess,
                model_name='124M',
                batch_size=1,
                prefix = "<Start Ep>",
      
                temperature=.7,
                truncate = "<End Ep>",
                include_prefix = True,
                destination_path=f'/content/drive/MyDrive/GPT2 SIMPLE FILTERED/class_false_filtered/spongebob_{i}.txt')


In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')
        

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below, comented out for safety:

In [ ]:
#!kill -9 -1